In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib

# 设置图像标签显示中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
# 关掉pandas的warnings
pd.options.mode.chained_assignment = None

# 导入数据

In [18]:
Bitcoin = pd.read_csv('Data\BCHAIN-MKPRU.csv')
Gold = pd.read_csv('Data\LBMA-GOLD.csv')
Bitcoin.columns = ['Date','Value']
Gold.columns = ['Date','Value']
Bitcoin['Date'] = pd.to_datetime(Bitcoin['Date'])
Gold['Date'] = pd.to_datetime(Gold['Date'])
all_date = Bitcoin['Date'].tolist()

In [19]:
init_date = all_date[0] # 起始日期
print(init_date)
interval = pd.Timedelta(20,unit='day') # 设置间隔日期
print(interval)

2016-09-11 00:00:00
20 days 00:00:00


## 关键定义

In [ ]:
def get_unit(data, N1=7, N2=7, N3=20, money=1000):
    # 起始资金
    money_init = money
    # 最近7个交易日的最高价
    High = talib.MAX(data['Value'], timeperiod=N1).shift(1)
    data['High'] = High
    # 最近7个交易日的最低价
    Low = talib.MIN(data['Value'], timeperiod=N2).shift(1)
    data['Low'] = Low
    # 真实波动幅度 True Range
    TR = talib.TRANGE(High, Low, data['Value'])
    data['TR'] = TR
    # 平均真实波幅
    ATR = talib.ATR(High, Low, data['Value'], timeperiod=N3)
    data['ATR'] = ATR
    # 头寸规模单位
    Unit = (money_init * 0.01) / ATR
    data['Unit'] = Unit
    return data

In [108]:
Bitcoin_Data = Bitcoin.copy()
Gold_Data = Gold.copy()
Bitcoin_Data = get_unit(Bitcoin_Data)
Gold_Data = get_unit(Gold_Data)
print(Bitcoin_Data.iloc[26:, :])
print(Gold_Data.iloc[26:, :])

           Date     Value      High       Low       TR          ATR      Unit
26   2016-10-07    617.21    614.82    607.18     7.64          NaN       NaN
27   2016-10-08    614.74    617.21    607.18    10.03    11.724000  0.852951
28   2016-10-09    615.65    617.21    607.18    10.03    11.639300  0.859158
29   2016-10-10    617.54    617.21    607.18    10.03    11.558835  0.865139
30   2016-10-11    614.77    617.54    607.18    10.36    11.498893  0.869649
...         ...       ...       ...       ...      ...          ...       ...
1821 2021-09-06  51769.06  50035.33  47074.77  2960.56  3637.213855  0.002749
1822 2021-09-07  52677.40  51769.06  47074.77  4694.29  3690.067663  0.002710
1823 2021-09-08  46809.17  52677.40  47155.87  5521.53  3781.640779  0.002644
1824 2021-09-09  46078.38  52677.40  46809.17  5868.23  3885.970241  0.002573
1825 2021-09-10  46368.69  52677.40  46078.38  6599.02  4021.622728  0.002487

[1800 rows x 7 columns]
           Date    Value     High      

## 测试

In [188]:
Bitcoin_Data = Bitcoin.copy()
High = talib.MAX(Bitcoin_Data['Value'], timeperiod=7).shift(1)
Low = talib.MIN(Bitcoin_Data['Value'], timeperiod=7).shift(1)
Bitcoin_Data['High'] = High
Bitcoin_Data['Low'] = Low
TR = talib.TRANGE(High, Low, Bitcoin_Data['Value'])
Bitcoin_Data['TR'] = TR
ATR = talib.ATR(High, Low, Bitcoin_Data['Value'], timeperiod=20)
Bitcoin_Data['ATR'] = ATR
Bitcoin_Data['upper_bound'] = Bitcoin_Data['Value'].shift(2).rolling(20).max()
Bitcoin_Data['lower_bound'] = Bitcoin_Data['Value'].shift(2).rolling(10).min()
Bitcoin_Data['Value Shift'] = Bitcoin_Data['Value'].shift(1)

In [189]:
# 启动回测 模拟
capital = 1000.0  # 初始资本价值（美元）
crypto = 0 # 加密的初始数量
fees = 0.02  # 佣金为2%
positions = []  # 保存交易位置
success_history = []  # 保存成功交易位置
failure_history = []  # 保持失败交易位置
print("-" * 50)
print("初始资本价值（美元）", capital)
print("-" * 50)

--------------------------------------------------
初始资本价值（美元） 1000.0
--------------------------------------------------


In [25]:
def get_unit(data, T=7, A=20, R1=20,R2=10,money=1000):
    # 起始资金
    money_init = money
    # 最近T个交易日的最高价
    data['High'] = talib.MAX(data['Value'], timeperiod=T).shift(1)
    # 最近T个交易日的最低价
    data['Low'] = talib.MIN(data['Value'], timeperiod=T).shift(1)
    # 通过以上数据近真实波动幅度 True Range
    data['TR'] = talib.TRANGE(data['High'], data['Low'], data['Value'])
    # 平均真实波幅
    data['ATR'] = talib.ATR(data['High'], data['Low'], data['Value'], timeperiod=A)
    # 头寸规模单位
    data['Unit'] = (money_init * 0.01) / data['ATR']
    data['upper_bound'] = data['Value'].shift(2).rolling(R1).max()
    data['lower_bound'] = data['Value'].shift(2).rolling(R2).min()
    data['Value Shift'] = data['Value'].shift(1)
    return data

In [34]:
def reverse(data):
    capital = 1000.0  # 初始资本价值（美元）
    crypto = 0 # 加密的初始数量
    fees = 0.02  # 佣金为2%
    positions = []  # 保存交易位置
    success_history = []  # 保存成功交易位置
    failure_history = []  # 保持失败交易位置
    for i in range(28, Bitcoin_Data.shape[0]):
        # 我们从28天开始循环，让ATR正确填充
    # 如果前一天的收盘价超过高价突破位，且未进场交易
    # 不考虑做空，进场仅买入
        if (data["Value Shift"].iloc[i] > data["upper_bound"].iloc[i] and len(positions) == 0):
            # 使用当前价格买入
            step = capital * (1.0 - fees)
            price = data['Value'].iloc[i]
            crypto += np.round(step / price, 4)
            capital = 0.0  # 满仓干
            stop_loss = price - 2.0 * data["ATR"].iloc[i]  # 止损价位
            positions += [{"Date": data['Date'].iloc[i], "Price": price}]
            print("交易位置: ", price, "持有份额", crypto, "交易时间", data['Date'].iloc[i], "止损价位", stop_loss)
        # 进场交易以后, 当价格低于低价突破位，或低于止损价位时, 或回测结束时
        elif len(positions) > 0 and (data["Value Shift"].iloc[i] < data["lower_bound"].iloc[i] or data["Value Shift"].iloc[i] < stop_loss or i == data.shape[0] - 1):
            # 使用当前价格卖出
            price = data['Value'].iloc[i]
            capital = crypto * price * (1 - fees)
            # 止损位和持仓归零
            stop_loss, crypto = 0.0, 0.0
            print("卖出价格", price, "资本", capital, "时间", data['Date'].iloc[i])
            # 记录交易成功或失败
            if positions[-1]["Price"] < price:
                for p in positions:
                    success_history += [{"Date": [p["Date"], data['Date'].iloc[i]],"Price": [p["Price"], price]}]
            else:
                for p in positions:
                    failure_history += [{"Date": [p["Date"], data['Date'].iloc[i]],"Price": [p["Price"], price],}]
            positions = []
    return success_history,failure_history

--------------------------------------------------
初始资本价值: 1000美元
--------------------------------------------------


In [39]:
# 启动回测 模拟
print("-" * 50)
print("初始资本价值: 1000美元")
print("-" * 50)
Bitcoin_Data = get_unit(Bitcoin.copy(), T=7, A=20, R1=20,R2=10,money=1000)
success_history,failure_history = reverse(Bitcoin_Data)


--------------------------------------------------
初始资本价值: 1000美元
--------------------------------------------------
交易位置:  614.77 持有份额 1.5941 交易时间 2016-10-11 00:00:00 止损价位 591.7722134999999
卖出价格 727.31 资本 1136.2167735799999 时间 2016-11-28 00:00:00
交易位置:  764.33 持有份额 1.4568 交易时间 2016-12-03 00:00:00 止损价位 710.2098971303462
卖出价格 908.14 资本 1296.51878496 时间 2017-01-08 00:00:00
交易位置:  979.703875 持有份额 1.2969 交易时间 2017-02-01 00:00:00 止损价位 813.6865602600217
卖出价格 952.2323625 资本 1210.251147907725 时间 2017-03-18 00:00:00
交易位置:  1181.149838 持有份额 1.0041 交易时间 2017-04-08 00:00:00 止损价位 911.84761704325
卖出价格 2464.959814 资本 2425.5648262526515 时间 2017-06-16 00:00:00
交易位置:  2807.609857 持有份额 0.8466 交易时间 2017-07-22 00:00:00 止损价位 2174.5348736519827
卖出价格 4312.26 资本 3577.7441296800002 时间 2017-09-09 00:00:00
交易位置:  4386.88375 持有份额 0.7992 交易时间 2017-10-02 00:00:00 止损价位 3325.369931549862
卖出价格 5716.301583 资本 4477.098860630928 时间 2017-11-12 00:00:00
交易位置:  7868.77 持有份额 0.5576 交易时间 2017-11-17 00:00:00 止损价位 6065.541117910

In [40]:
s = len(success_history)
f = len(failure_history)
print("交易成功率: {}%".format(s/(s+f)*100))

交易成功率: 59.375%


In [41]:
buy_date = []
sell_date = []
buy_price = []
sell_price = []

for i in range(len(success_history)):
    buy_date.append(success_history[i]['Date'][0])
    sell_date.append(success_history[i]['Date'][1])
    buy_price.append(success_history[i]['Price'][0])
    sell_price.append(success_history[i]['Price'][1])

for i in range(len(failure_history)):
    buy_date.append(failure_history[i]['Date'][0])
    sell_date.append(failure_history[i]['Date'][1])
    buy_price.append(failure_history[i]['Price'][0])
    sell_price.append(failure_history[i]['Price'][1])

buy_price = pd.Series(buy_price).tolist()
buy_date = pd.Series(buy_date).tolist()
sell_price = pd.Series(sell_price).tolist()
sell_date = pd.Series(sell_date).tolist()

#buy_price = np.array(buy_price).reshape(-1, 1)
#buy_date = np.array(buy_date).reshape(-1, 1)
#sell_price = np.array(sell_price).reshape(-1, 1)
#sell_date = np.array(sell_date).reshape(-1, 1)
#buy = np.hstack((buy_date, buy_price))
#sell = np.hstack((sell_date, sell_price))

In [42]:
from pyecharts.charts import Line,EffectScatter
import pyecharts.options as opts

In [43]:
line = Line(init_opts=opts.InitOpts(width="1400px", height="600px"))

date = Bitcoin_Data.iloc[:, 0].tolist()
data_close = Bitcoin_Data.Value.tolist()

line.add_xaxis(xaxis_data=date)

line.add_yaxis(
    series_name="收盘价",
    y_axis=data_close,
    is_connect_nones=False,  # 是否连接空数据
    is_smooth=True,  # 是否平滑曲线
    is_symbol_show=True)


#添加买卖信号
es = EffectScatter()
es.add_xaxis(xaxis_data=sell_date)
es.add_yaxis(
    series_name="sell",
    y_axis=sell_price,
)

es.add_xaxis(xaxis_data=buy_date)
es.add_yaxis(
    series_name="buy",
    y_axis=buy_price,
    symbol="triangle",
)

line.overlap(es)
line.render_notebook()